In [1]:
import pandas as pd
import numpy as np
import seaborn as sns; sns.set()
import matplotlib as mpl
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import urllib.request
from tqdm.notebook import tqdm

In [2]:
url = requests.get('https://www.bfi.org.uk/films-tv-people/sightandsoundpoll2012/voters')

In [3]:
soup = BeautifulSoup(url.content, 'html.parser')

In [5]:
soup.findAll('tr')

[<tr>
 <td><p><a href="https://www2.bfi.org.uk/films-tv-people/sightandsoundpoll2012/voter/558">Gulnara Abikeyeva</a></p></td>
 <td><p>programmer</p></td>
 <td><p>Kazakhstan</p></td>
 <td><p>Female</p></td>
 </tr>,
 <tr>
 <td><p><a href="https://www2.bfi.org.uk/films-tv-people/sightandsoundpoll2012/voter/1033">Lenny Abrahamson</a></p></td>
 <td><p>Director</p></td>
 <td><p>Ireland</p></td>
 <td><p>Male</p></td>
 </tr>,
 <tr>
 <td><p><a href="https://www2.bfi.org.uk/films-tv-people/sightandsoundpoll2012/voter/316">Mehmet Açar</a></p></td>
 <td><p>critic</p></td>
 <td><p>Turkey</p></td>
 <td><p>Male</p></td>
 </tr>,
 <tr>
 <td><p><a href="https://www2.bfi.org.uk/films-tv-people/sightandsoundpoll2012/voter/1090">Newton Aduaka</a></p></td>
 <td><p>Director</p></td>
 <td><p>Nigerla</p></td>
 <td><p>Male</p></td>
 </tr>,
 <tr>
 <td><p><a href="https://www2.bfi.org.uk/films-tv-people/sightandsoundpoll2012/voter/624">Eva af Geijerstam</a></p></td>
 <td><p>critic</p></td>
 <td><p>Sweden</p></td

In [242]:
voters = soup.findAll('tr')

In [243]:
name_l, profession_l, country_l, gender_l, link_l = [], [], [], [], []

for i in range(len(voters)):
    info = voters[i].findAll('p')
    name_l.append(info[0].text)
    profession_l.append(info[1].text)
    country_l.append(info[2])
    gender_l.append(info[3].text)
    link_l.append(info[0].find('a').get('href'))

In [244]:
voter_map = pd.DataFrame({'name':name_l, 
                          'profession':profession_l, 
                          'country':country_l, 
                          'gender':gender_l, 
                          'link':link_l})

In [245]:
voter_map['country'] = voter_map['country'].astype(str).str[3:-4]

In [246]:
voter_map.to_csv('../data/voter_map.csv', index=False)

In [248]:
name_l, profession_l, voter_country_l, movie_l, year_l, director_l, movie_link_l = [], [], [], [], [], [], []

for x in tqdm(range(voter_map.shape[0])):
    url = requests.get(voter_map['link'][x])
    soup = BeautifulSoup(url.content, 'html.parser')
    soup = soup.find(class_ = 'sas-poll')
    top10 = soup.findAll('tr')
    
    name = voter_map['name'][x]
    profession = voter_map['profession'][x]
    voter_country = voter_map['country'][x]

    for k in range(len(top10)):
        movie = top10[k].findAll('p')[0].text
        year = top10[k].findAll('p')[1].text
        director = top10[k].findAll('p')[2].text
        
        if (top10[k].find('a')==None):
            movie_link = 'na'
        else:
            movie_link = top10[k].findAll('a')[0].get('href')

        name_l.append(name)
        profession_l.append(profession)
        movie_l.append(movie)
        year_l.append(year)
        director_l.append(director)
        movie_link_l.append(movie_link)
        voter_country_l.append(voter_country)

In [273]:
new_df=pd.DataFrame({'voter':name_l, 
                     'voter_profession':profession_l,
                     'voter_country':voter_country_l,
                     'movie':movie_l,
                     'year':year_l,
                     'director':director_l,
                     'movie_link':movie_link_l})

In [275]:
new_df.loc[new_df['voter_profession']!='Director', 'critic']=1
new_df['critic']=new_df['critic'].fillna(0)

In [276]:
new_df.loc[(new_df['movie']=='Street of Crocodiles') & 
           (new_df['director']=='Brothers Quay'), 'director'] = 'Quay Brothers'

new_df.loc[(new_df['movie']=='Yol') & 
           (new_df['director']=='Yilmaz Güney'), 'year'] = 1982
new_df.loc[(new_df['movie']=='Yol') & 
           (new_df['director']=='Yilmaz Güney'), 'director'] = 'Serif Gören/Yilmaz Güney'

new_df.loc[new_df['movie']=='Atanarjuat The Fast Runner', 'year'] = 2000
new_df.loc[new_df['director']=='Zacharius Kunuk', 'director'] = 'Zacharias Kunuk'



new_df.loc[(new_df['movie']=='Tale of the Wind, A') & 
           (new_df['director']=='Joris Ivens'), 'director'] = 'Joris Ivens/Marceline Loridan Ivens'


In [277]:
movie_xwalk = new_df[['movie', 'year', 'director']].drop_duplicates().reset_index(drop=True)
movie_xwalk['movie_id'] = 'id_' + movie_xwalk.index.astype(str)
movie_xwalk.to_csv('../data/movie_xwalk.csv', index=False)

In [278]:
new_df = pd.merge(new_df, movie_xwalk, how='left', on=['movie', 'year', 'director'])
new_df[['voter', 'voter_profession', 'voter_country', 'movie_id']].to_csv('../data/all_votes.csv', index=False)

## Scraping BFI Information

In [286]:
movie_links = new_df[['movie_id', 'movie_link']].drop_duplicates().reset_index(drop=True)
movie_links = movie_links[movie_links['movie_link']!='na'].reset_index(drop=True)

In [289]:
df = pd.DataFrame()
for i in tqdm(range(movie_links.shape[0])):
    url = requests.get(movie_links['movie_link'][i])
    soup = BeautifulSoup(url.content, 'html.parser')
    table = soup.findAll(class_ = 'table-items')[0]
    variable_l , content_l = [], []

    for j in range(len(table.findAll('li'))):    
        variable_l.append(table.findAll('li')[j].find(class_='row-label').text)

        elements = []
        for elem in table.findAll('li')[j].findAll(class_='secondary label'):
            elements.append(elem.text)
        elements = ', '.join(elements)

        content_l.append(elements)
        
    temp = pd.DataFrame({0:content_l}).T
    temp.columns = variable_l
    
    temp.rename(columns={'Countries':'Country', 
                         'Directors':'Director', 
                         'Genres':'Genre'}, inplace = True)
    temp['movie_id'] = movie_links['movie_id'][i]
        
    df = pd.concat([df, temp], axis = 0, sort=False)

IndexError: list index out of range

In [ ]:
df.to_csv('../data/bfi_information.tsv', sep='\t', index=False)

# Sandbox

In [39]:
url = requests.get(movie_map['movie_link'][4])

In [65]:
soup = BeautifulSoup(url.content, 'html.parser')

In [67]:
table = soup.findAll(class_ = 'table-items')[0]

In [104]:
variable_l , content_l = [], []

for i in range(len(table.findAll('li'))):    
    variable_l.append(table.findAll('li')[i].find(class_='row-label').text)

    elements = []
    for elem in table.findAll('li')[i].findAll(class_='secondary label'):
        elements.append(elem.text)
    elements = ', '.join(elements)

    content_l.append(elements)

In [105]:
variable_l

['Featuring', 'Director', 'Country', 'Year', 'Genre', 'Type', 'Category']

In [107]:
content_l

['Charles Chaplin, Paulette Goddard, Henry Bergman',
 'Charles Chaplin',
 'USA',
 '1936',
 'Comedy',
 'Film',
 'Fiction']

In [114]:
temp = pd.DataFrame({0:content_l}).T
temp.columns = variable_l

In [118]:
temp

,Featuring,Director,Country,Year,Genre,Type,Category
0,"Charles Chaplin, Paulette Goddard, Henry Bergman",Charles Chaplin,USA,1936,Comedy,Film,Fiction


In [71]:
for i in range(table.findAll('li')):
    variable = 

[<li>
 <p class="row-label">Featuring</p>
 <p>
 <a class="secondary label" href="https://www.bfi.org.uk/films-tv-people/4ce2b9efe2680">Charles Chaplin</a>
 <a class="secondary label" href="https://www.bfi.org.uk/films-tv-people/4ce2b9f3b34bd">Paulette Goddard</a>
 <a class="secondary label" href="https://www.bfi.org.uk/films-tv-people/4ce2ba1eb2d62">Henry Bergman</a>
 </p>
 </li>,
 <li>
 <p class="row-label">Director</p>
 <p>
 <a class="secondary label" href="https://www.bfi.org.uk/films-tv-people/4ce2b9efe2680">Charles Chaplin</a>
 </p>
 </li>,
 <li>
 <p class="row-label">Country</p>
 <p>
 <span class="secondary label">USA</span>
 </p>
 </li>,
 <li>
 <p class="row-label">Year</p>
 <p>
 <span class="secondary label">1936</span>
 </p>
 </li>,
 <li>
 <p class="row-label">Genre</p>
 <p>
 <span class="secondary label">Comedy</span>
 </p>
 </li>,
 <li>
 <p class="row-label">Type</p>
 <p>
 <span class="secondary label">Film</span>
 </p>
 </li>,
 <li>
 <p class="row-label">Category</p>
 <p>
 

In [51]:
soup[2]

<p class="row-label">Country</p>

In [52]:
soup.find('Country')

AttributeError: ResultSet object has no attribute 'find'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [ ]:
<p class="row-label">Country</p>

In [46]:
soup

<!DOCTYPE html>

<!-- Sorry no IE7 support! -->
<!-- @see http://foundation.zurb.com/docs/index.html#basicHTMLMarkup -->
<!--[if IE 8]><html class="no-js lt-ie9" lang="en"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en"> <!--<![endif]-->
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="Drupal 7 (http://drupal.org)" name="Generator"/>
<meta content="https://www.bfi.org.uk/films-tv-people/4ce2b6bbbf1e2" property="og:url"/>
<meta content="BFI" property="og:site_name"/>
<meta content="190731001052619" property="fb:app_id"/>
<meta content="https://www.bfi.org.uk/films-tv-people/sites/bfi.org.uk.films-tv-people/files/image/modern-times-bfi-00o-5h9.jpg" property="og:image"/>
<meta content="Modern Times (1936)" property="og:title"/>
<meta content="The final outing for Charlie Chaplin&amp;rsquo;s beloved Tramp character finds him enduring the pratfalls and humiliations of work in an increasingly mechanised society." property="og:descr

In [ ]:
<p class="row-label">Country</p>

## Importing 

In [358]:
movie_map[movie_map['director'].str.contains('/')]

,movie,year,director,movie_link,total_score,critic_score,director_score,french_voters_score
3,"Matrix, The",1999,Andy Wachowski/Lana Wachowski,https://www.bfi.org.uk/films-tv-people/4ce2b81...,6,5.0,1.0,NaN
48,"Son, The",2002,Jean-Pierre Dardenne/Luc Dardenne,https://www.bfi.org.uk/films-tv-people/4ce2b87...,5,3.0,2.0,NaN
68,Red River,1947,Howard Hawks/Arthur Rosson,https://www.bfi.org.uk/films-tv-people/4ce2b6b...,10,7.0,3.0,NaN
91,Not Reconciled,1965,Jean-Marie Straub/Danièle Huillet,https://www.bfi.org.uk/films-tv-people/4ce2b74...,3,1.0,2.0,NaN
92,Numéro deux,1975,Jean-Luc Godard/Anne-Marie Miéville,https://www.bfi.org.uk/films-tv-people/4ce2b6b...,6,5.0,1.0,NaN
...,...,...,...,...,...,...,...,...
2491,"Vampires of Poverty, The",1978,Carlos Mayolo/Luis Ospina,https://www.bfi.org.uk/films-tv-people/4ce2b6e...,1,1.0,NaN,NaN
2492,Whisky,2004,Pablo Stoll/Juan Pablo Rebella,https://www.bfi.org.uk/films-tv-people/4ce2b8a...,1,1.0,NaN,NaN
2514,On the Town,1949,Stanley Donen/Gene Kelly,https://www.bfi.org.uk/films-tv-people/4ce2b6b...,1,1.0,NaN,NaN
2525,"Laughing Man, The",1967,Walter Heynowski/Gerhard Scheumann,https://www.bfi.org.uk/films-tv-people/4ce2b69...,1,1.0,NaN,NaN


In [359]:
movie_map[movie_map['movie'].str.contains('Empire')]

,movie,year,director,movie_link,total_score,critic_score,director_score,french_voters_score
284,Inland Empire,2006,David Lynch,https://www.bfi.org.uk/films-tv-people/4ce2b8b...,7,6.0,1.0,NaN
1016,Empire,1964,Andy Warhol,https://www.bfi.org.uk/films-tv-people/4ce2b71...,6,3.0,3.0,NaN
1773,"Empire Strikes Back, The",1980,Irvin Kershner,https://www.bfi.org.uk/films-tv-people/4ce2b6b...,3,2.0,1.0,NaN


In [362]:
new_df[new_df['director']=='Melancholia']

,voter,profession,voter_country,movie,year,director,movie_link,critic
335,Nigel Andrews,critic,UK,Melancholia,2011,Lars von Trier,https://www.bfi.org.uk/films-tv-people/4f4b8d1...,1.0
475,Kutlug Ataman,Director,Turkey,Melancholia,2011,Lars von Trier,https://www.bfi.org.uk/films-tv-people/4f4b8d1...,0.0
964,Stig Björkman,critic,Sweden,Melancholia,2011,Lars von Trier,https://www.bfi.org.uk/films-tv-people/4f4b8d1...,1.0
1436,Martin Tudor Caranfil,critic,Romania,Melancholia,2011,Lars von Trier,https://www.bfi.org.uk/films-tv-people/4f4b8d1...,1.0
5090,Jytte Jensen,archivist,US,Melancholia,2011,Lars von Trier,https://www.bfi.org.uk/films-tv-people/4f4b8d1...,1.0
6654,Peter Machen,critic,South Africa,Melancholia,2011,Lars von Trier,https://www.bfi.org.uk/films-tv-people/4f4b8d1...,1.0
7826,Shirin Neshat,Director,Iran/US,Melancholia,2011,Lars von Trier,https://www.bfi.org.uk/films-tv-people/4f4b8d1...,0.0
7894,Angelina Nikonova,Director,Russia,Melancholia,2011,Lars von Trier,https://www.bfi.org.uk/films-tv-people/4f4b8d1...,0.0
8226,Laila Pakalnina,Director,Latvia,Melancholia,2011,Lars von Trier,https://www.bfi.org.uk/films-tv-people/4f4b8d1...,0.0
10186,David Sorfa,academic,UK,Melancholia,2011,Lars von Trier,https://www.bfi.org.uk/films-tv-people/4f4b8d1...,1.0


In [ ]:
new_df['director'] = new_df[2].apply(lambda x: x[:x.find(',')])

In [378]:
movie_map['director'].apply(lambda x: x[:x.find('/')])

0            Aktan Arym Kuba
1           Mohsen Makhmalba
2              Shaken Aimano
3             Andy Wachowski
4             Charles Chapli
                ...         
2562    Goffredo Alessandrin
2563             Nicholas Ra
2564      Pier Paolo Pasolin
2565         Laurence Olivie
2566              Louis Mall
Name: director, Length: 2567, dtype: object

In [366]:
movie_map[movie_map['director'].str.contains('Trier')]

,movie,year,director,movie_link,total_score,critic_score,director_score,french_voters_score
98,Breaking the Waves,1996,Lars von Trier,https://www.bfi.org.uk/films-tv-people/4ce2b7e...,15,9.0,6.0,NaN
247,Melancholia,2011,Lars von Trier,https://www.bfi.org.uk/films-tv-people/4f4b8d1...,12,7.0,5.0,NaN
878,"Idiots, The",1998,Lars von Trier,https://www.bfi.org.uk/films-tv-people/4ce2b80...,7,4.0,3.0,NaN
986,"Five Obstructions, The",2003,Jørgen Leth/Lars von Trier,https://www.bfi.org.uk/films-tv-people/4ce2b87...,1,1.0,NaN,NaN
1002,"Kingdom, The",1997,Lars von Trier,https://www.bfi.org.uk/films-tv-people/4ce2b7d...,2,2.0,NaN,NaN
1160,Dogville,2003,Lars von Trier,https://www.bfi.org.uk/films-tv-people/4ce2b85...,9,6.0,3.0,NaN
1199,Europa,1991,Lars von Trier,https://www.bfi.org.uk/films-tv-people/4ce2b7a...,2,2.0,NaN,NaN
1209,Dancer in the Dark,2000,Lars von Trier,https://www.bfi.org.uk/films-tv-people/4ce2b81...,4,NaN,4.0,NaN
1874,Epidemic,1987,Lars von Trier,https://www.bfi.org.uk/films-tv-people/4ce2b71...,1,1.0,NaN,NaN
2094,Manderlay,2005,Lars von Trier,https://www.bfi.org.uk/films-tv-people/4ce2b8a...,1,NaN,1.0,NaN


In [148]:
name_l, profession_l, movie_l, year_l, director_l = [], [], [], [], []


name = votes['name'][0]
profession = votes['profession'][0]

for k in range(len(top10)):
    movie = top10[k].findAll('p')[0].text
    year = top10[k].findAll('p')[1].text
    director = top10[k].findAll('p')[2].text

    name_l.append(name)
    profession_l.append(profession)
    movie_l.append(movie)
    year_l.append(year)
    director_l.append(year)

In [149]:
movie_l

['Beshkempir',
 'Gabbeh',
 'Land of Fathers',
 'Matrix, The',
 'Modern Times',
 'Nights of Cabiria',
 'Rashomon',
 'Solaris',
 'Spring, Summer, Fall, Winter and Spring…',
 'Wild Strawberries']

In [135]:
critic

'Gulnara Abikeyeva'

In [138]:
votes

,name,profession,country,gender,link
0,Gulnara Abikeyeva,programmer,[Kazakhstan],Female,https://www.bfi.org.uk/films-tv-people/sightan...
1,Lenny Abrahamson,Director,[Ireland],Male,https://www.bfi.org.uk/films-tv-people/sightan...
2,Mehmet Açar,critic,[Turkey],Male,https://www.bfi.org.uk/films-tv-people/sightan...
3,Newton Aduaka,Director,[Nigerla],Male,https://www.bfi.org.uk/films-tv-people/sightan...
4,Eva af Geijerstam,critic,[Sweden],Female,https://www.bfi.org.uk/films-tv-people/sightan...
...,...,...,...,...,...
1200,Marco Ettore Zucchi,critic,[Switzerland],Male,https://www.bfi.org.uk/films-tv-people/sightan...
1201,Gertjan Zuilhof,programmer,[Netherlands],Male,https://www.bfi.org.uk/films-tv-people/sightan...
1202,Andrzej Żuławski,Director,[Poland],Male,https://www.bfi.org.uk/films-tv-people/sightan...
1203,Santos Zunzunegui,academic,[Spain],Male,https://www.bfi.org.uk/films-tv-people/sightan...
